In [18]:
# S1: iterate over all the 10-k files in the directory
import re
from pathlib import Path
import pandas as pd


"""
Step 1: Iterate over all the 10-k files in the directory
"""
current_dictionary = Path.cwd()
# TODO: Path.cwd() or Path('A1/10-k html files')
for html_file in current_dictionary.glob('*.html'):
    # TODO: utf-8 encoding or ISO-8859-1 encoding or ASCII?
    with html_file.open('r', encoding='ASCII') as file:
        # print(file.read()[0:100])
        # read or readlines? Which one is more space efficient?
        raw_content = file.read()

In [19]:

"""
Step 2: Extract all necessary information from the 10-k files
"""
def clean_html(raw_html):
    """
    This function is used to clean the raw content
    """
    # remove html tags
    clean_content = re.sub(r'<.*?>', ' ', raw_html)
    # remove html entities
    clean_content = re.sub(r'&\w+;', ' ', clean_content)
    # remove multiple spaces
    clean_content = re.sub(r'\s+', ' ', clean_content)
    return clean_content